In [ ]:
import csv
import pandas as pd
import time
tweets=[]
start= time.time()
data= pd.read_csv('data/training.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
print(data[:5])
print('time',time.time()-start)
print(data.shape)

In [ ]:

new_df = data.drop(['id','date','query','user'], axis=1)
tweets = new_df.set_index('Type')
print(tweets[-10:])

In [ ]:
tweets_neg=tweets.loc[[0]]
tweets_pos=tweets.loc[[4]]
# print(tweets_neg[-0:])
# print(tweets_pos[0:10])
print(tweets_pos[['tweet']])

In [ ]:
dict_neg_words={}
tweets_neg['words']= tweets_neg['tweet'].str.split()
tweets_neg['word_count']= tweets_neg['words'].str.len()
tweets_pos['words']= tweets_pos['tweet'].str.split()
tweets_pos['word_count']= tweets_pos['words'].str.len()
print(tweets_neg[:5])
print(tweets_pos[:5])
print('Done')

In [ ]:
dict_neg_words={}
for l_words in tweets_neg['words']:
    for word in l_words:
        word=word.lower()
        if word in dict_neg_words:
            dict_neg_words[word] +=1
        else:
            dict_neg_words[word] =1
print(dict_neg_words)

In [ ]:
phi_neg = tweets_neg.shape[0]/tweets.shape[0]
phi_pos =tweets_pos.shape[0]/tweets.shape[0]